store image global feature into lmdb database

In [1]:
import os
import sys
caffe_root = os.environ['CAFFE_PROJ_DIR']
sys.path.append(caffe_root + 'python')
import caffe
sys.path.append(caffe_root + 'python/util')
from util_func import *
sys.path.append(caffe_root + 'python/caffe/proto')
import caffe_pb2

import numpy as np
import lmdb

# caffe_local_root = '/home/zyan3/local/proj/caffe_private_hdcnn/'

print 'caffe_root: %s' % caffe_root
# print 'caffe_local_root: %s' % caffe_local_root


caffe_root: /home/zyan3/proj/caffe_private_video_enhance/


In [2]:
dl_img_proj_root='/home/zyan3/proj/dl-image-enhance/'
video_enhance_proj_root='/home/zyan3/proj/caffe_private_video_enhance/video-enhance/'
global_ftr_dir = dl_img_proj_root + 'data/uniform_set/globalFtr/'
global_ftr_lmdb_fn=video_enhance_proj_root + 'data/uniform_set/global_ftr_lmdb/'

In [3]:

global_ftr_lmdb= lmdb.open(global_ftr_lmdb_fn)
global_ftr_txn=global_ftr_lmdb.begin(write=True)

In [4]:
global_ftr_fns=os.listdir(global_ftr_dir)
print len(global_ftr_fns)

115


In [5]:
for i,global_ftr_fn in enumerate(global_ftr_fns):
    global_ftr_dict=unpickle(global_ftr_dir+global_ftr_fn)
    global_ftr=global_ftr_dict['pix_global_ftr']    
    datum = caffe_pb2.Datum()
    datum.channels = global_ftr.shape[0]
    datum.height = 1
    datum.width = 1
    
    for j in range(global_ftr.shape[0]):
        datum.float_data.append(global_ftr[j])    
    key=global_ftr_fn
    if i==0:
        print key
    global_ftr_txn.put(key,datum.SerializeToString())

3124621920_4ac3d9ea81_z


In [6]:
global_ftr_cursor=global_ftr_txn.cursor()
count = 0
# print '%d keys' % len(prob_cursor.keys())
for key, value in global_ftr_cursor:
    if count < 10:
        print key
    count += 1
print 'count:%d' % count

10589229553_c1c8d3f54a_z
10656328515_670c22a79a_z
1098157481_691e4df198_z
11144766034_7dc7162505_z
12506493305_e6ed54a2ba_z
12852914404_cabcf1ffca_z
1298860319_c925153751_z
13090677244_fdf2d6ba58_z
1315958474_e5add54fc4_z
13373338193_4cacbbc487_z
count:115


In [7]:
global_ftr_txn.commit()
global_ftr_lmdb.close()

generate feature mean files in format binaryproto

In [9]:
train_batch_meta=unpickle(dl_img_proj_root + 'data/uniform_set_xpro/uniform_set_xpro_7K_10_batch_seg_voting_k0.10/batches.meta')
print train_batch_meta['global_ftr_mean'].shape
print train_batch_meta['data_mean'].shape
print train_batch_meta['pixContextSemMean'].shape

(237,)
(3,)
(500,)


In [29]:
global_ftr_mean_blob = caffe_pb2.BlobProto()
global_ftr_mean_blob.num = 1
global_ftr_mean_blob.channels = train_batch_meta['global_ftr_mean'].shape[0]
global_ftr_mean_blob.height = 1
global_ftr_mean_blob.width = 1
for i in range(train_batch_meta['global_ftr_mean'].shape[0]):
    global_ftr_mean_blob.data.append(float(train_batch_meta['global_ftr_mean'][i]))

f = open(video_enhance_proj_root + 'data/uniform_set/global_feature_mean.binaryproto', "wb")
f.write(global_ftr_mean_blob.SerializeToString())
f.close()

In [30]:
print video_enhance_proj_root

/home/zyan3/proj/caffe_private_video_enhance/video-enhance/


In [31]:
pixel_ftr_mean_blob = caffe_pb2.BlobProto()
pixel_ftr_mean_blob.num = 1
pixel_ftr_mean_blob.channels = train_batch_meta['data_mean'].shape[0]
pixel_ftr_mean_blob.height = 1
pixel_ftr_mean_blob.width = 1
for i in range(train_batch_meta['data_mean'].shape[0]):
    pixel_ftr_mean_blob.data.append(float(train_batch_meta['data_mean'][i]))

f = open(video_enhance_proj_root + 'data/uniform_set/pixel_feature_mean.binaryproto', "wb")
f.write(pixel_ftr_mean_blob.SerializeToString())
f.close()

In [4]:
'''mean files for semantic context features'''
local_caffe_private_hdcnn_dir='/home/zyan3/local/proj/caffe_private_hdcnn/'
semantic_context_feature_dirs=\
[local_caffe_private_hdcnn_dir+'examples/uniform_set/VGG_ILSVRC_16_layers_parser_fc7/stitch_fc7_longside_1024/',\
 local_caffe_private_hdcnn_dir+'examples/uniform_set/VGG_ILSVRC_16_layers_parser_fc7/stitch_fc7_longside_2048/']

In [8]:
n_train_img, semantic_ftr_dim=115,4096
semantic_context_feature_scale_num = len(semantic_context_feature_dirs)
semantic_ftr_means=np.zeros((semantic_context_feature_scale_num,semantic_ftr_dim),
                            dtype=np.single)
for i, semantic_context_feature_dir in enumerate(semantic_context_feature_dirs):
    for j in range(n_train_img):
        if j%10==0:
            print '%d out of %d' % (j,n_train_img)
        meta_file=semantic_context_feature_dir+'%d.txt'%j
        data_file=semantic_context_feature_dir+'%d.dat'%j
        f=open(meta_file,'r')
        lines=f.readlines()
        channels=int(lines[0][:-1])
        height=int(lines[1][:-1])
        width=int(lines[2][:-1])
        f.close()
        
        
        f=open(data_file,'rb')
        ftr=np.fromfile(f,dtype=np.single)
        f.close()
        ftr=ftr.reshape((channels,height,width))
        ftr_mean = ftr.mean(axis=2).mean(axis=1)
        semantic_ftr_means[i,:] += ftr_mean
    semantic_ftr_means[i,:] /= n_train_img
print semantic_ftr_means[:,:10]

semantic_ftr_means=semantic_ftr_means.reshape((semantic_context_feature_scale_num*semantic_ftr_dim))

semantic_ftr_mean_blob = caffe_pb2.BlobProto()
semantic_ftr_mean_blob.num = 1
semantic_ftr_mean_blob.channels = semantic_ftr_means.shape[0]
semantic_ftr_mean_blob.height = 1
semantic_ftr_mean_blob.width = 1
for i in range(semantic_ftr_means.shape[0]):
    semantic_ftr_mean_blob.data.append(float(semantic_ftr_means[i]))

f = open(video_enhance_proj_root + 'data/uniform_set/semantic_context_feature_mean.binaryproto', "wb")
f.write(semantic_ftr_mean_blob.SerializeToString())
f.close()

0 out of 115
10 out of 115
20 out of 115
30 out of 115
40 out of 115
50 out of 115
60 out of 115
70 out of 115
80 out of 115
90 out of 115
100 out of 115
110 out of 115
0 out of 115
10 out of 115
20 out of 115
30 out of 115
40 out of 115
50 out of 115
60 out of 115
70 out of 115
80 out of 115
90 out of 115
100 out of 115
110 out of 115
[[-0.55568993  0.02073335  0.35535905 -0.2628513  -0.29997554 -0.37475386
  -0.03434702 -1.1003859  -0.94374883 -0.80052316]
 [-1.11279786  0.36140871  0.63885087 -0.18585218 -0.78360617 -0.24579981
   0.62798053 -2.13138771 -1.21467137 -1.45194674]]
